In [1]:
import json
import os
import random
from glob import glob
from tqdm import tqdm

In [2]:
files = glob('translated-LLaVA-Instruct-150K/*.jsonl')
files

['translated-LLaVA-Instruct-150K/detail_23k.translated.jsonl',
 'translated-LLaVA-Instruct-150K/conversation_58k.translated.jsonl',
 'translated-LLaVA-Instruct-150K/complex_reasoning_77k.translated.jsonl']

In [3]:
roles = {
    'human': 'user',
    'gpt': 'assistant'
}

In [11]:
data_en, data_ms = [], []
for f in files:
    with open(f) as fopen:
        for l in tqdm(fopen):
            l = json.loads(l)
            f = os.path.join('/home/ubuntu/translated-LLaVA-Instruct-150K/filtered-llava-images', l['image'])
            if not os.path.exists(f):
                continue
            en, ms = [], []
            for c in l['conversations']:
                en_ = c['value'].replace('<imej>','<image>').replace('<img>','<image>').replace('<gambar>','<image>')
                ms_ = c['value_ms'].replace('<imej>','<image>').replace('<img>','<image>').replace('<gambar>','<image>')
                
                if roles[c['from']] == 'user' and ('<image>' not in en_ or '<image>' not in ms_):
                    break
                    
                en.append({
                    'role': roles[c['from']],
                    'content': en_
                })
                ms.append({
                    'role': roles[c['from']],
                    'content': ms_
                })
            
            if len(en):
                data_en.append({'filename': [f], 'conversations': en})
            if len(ms):
                data_ms.append({'filename': [f], 'conversations': ms})

23240it [00:00, 47663.59it/s]
56681it [00:01, 30096.15it/s]
76643it [00:01, 42553.34it/s]


In [12]:
a = set()
for l in data_en + data_ms:
    for i in range(len(l['conversations'])):
        splitted = l['conversations'][i]['content'].split('<')
        if len(splitted) > 1:
            a.add(splitted[1].strip().split('\n')[0])
a

{'image>'}

In [13]:
len(data_en), len(data_ms)

(148657, 148657)

In [15]:
data_ms[2]

{'filename': ['/home/ubuntu/translated-LLaVA-Instruct-150K/filtered-llava-images/000000281721.jpg'],
 'conversations': [{'role': 'user',
   'content': 'Terangkan imej berikut.\n<image>'},
  {'role': 'assistant',
   'content': 'Imej itu memaparkan jalan bandar yang sibuk dengan kereta yang memandu di jalan raya dan pejalan kaki berjalan di sepanjang kaki lima. Pelbagai kenderaan melalui lampu hijau di persimpangan, bergerak ke kedua-dua arah. Sesetengah kereta lebih dekat dengan kamera, manakala yang lain boleh dilihat lebih jauh di jalan.\n\nTerdapat beberapa pejalan kaki yang kelihatan berhampiran bahagian kiri imej, mungkin menavigasi jalan mereka melalui bandar. Pelbagai lampu isyarat diletakkan di seluruh tempat kejadian, memastikan aliran trafik yang selamat untuk kedua-dua kenderaan dan pejalan kaki. Suasana menunjukkan persekitaran bandar yang sibuk dan meriah.'}]}

In [16]:
with open('prepared-llava-en.jsonl', 'w') as fopen:
    for d in data_en:
        fopen.write(f'{json.dumps(d)}\n')

In [17]:
with open('prepared-llava-ms.jsonl', 'w') as fopen:
    for d in data_ms:
        fopen.write(f'{json.dumps(d)}\n')